In [15]:
import json

with open("hubspot_benchmark.json") as f:
    bench_data = json.load(f)
    
bench_data

[{'intent': 'John needs to retrieve past engagements (calls, emails, meetings) related to Clara Reynolds to prepare the CEO for an important meeting.',
  'expected_tool_calls': [{'tool_name': 'hubspot__list-or-search-contacts__ListOrSearchContacts',
    'tool_input': '{"search":"Clara Reynolds"}',
    'tool_output': {'status': 'completed'}},
   {'tool_name': 'hubspot__list-or-search-engagements__ListOrSearchEngagements',
    'tool_input': '{"type": "CALL", "contactId": "<CONTACT_ID>"}',
    'tool_output': {'status': 'completed'}},
   {'tool_name': 'hubspot__list-or-search-engagements__ListOrSearchEngagements',
    'tool_input': '{"type": "EMAIL", "contactId": "<CONTACT_ID>"}',
    'tool_output': {'status': 'completed'}},
   {'tool_name': 'hubspot__list-or-search-engagements__ListOrSearchEngagements',
    'tool_input': '{"type": "MEETING", "contactId": "<CONTACT_ID>"}',
    'tool_output': {'status': 'completed'}},
   {'tool_name': 'hubspot__list-or-search-engagements__ListOrSearchEngage

In [16]:
import json
import sys
import os
import importlib
from pathlib import Path
from dotenv import load_dotenv

# Add the current directory to the path to access the benchmarks module
notebook_path = Path().absolute()
sys.path.append(str(notebook_path.parent))

import benchmarks.superface_client
importlib.reload(benchmarks.superface_client)
from benchmarks.superface_client import get_configuration_url

# Load environment variables from .env file
load_dotenv()

# Import the Superface client
import superface.client
importlib.reload(superface.client)
from superface.client import Superface

# Initialize Superface
superface = Superface(
    api_key=os.getenv("SUPERFACE_API_KEY")
)

# Get user ID from environment variables
user_id = os.getenv("SUPERFACE_BENCHMARK_USER_ID")
if not user_id:
    raise ValueError("SUPERFACE_BENCHMARK_USER_ID environment variable is required")

# Extract unique tools from benchmark data
unique_tools = set()
for test_case in bench_data:
    for expected_call in test_case.get('expected_tool_calls', []):
        tool_name = expected_call.get('tool_name')
        if tool_name:
            unique_tools.add(tool_name)

print(f"Found {len(unique_tools)} unique tools in benchmark data:")
for tool in unique_tools:
    print(f"- {tool}")

# Check if each tool is connected
all_tools_connected = True
tool_connection_status = {}

for tool in unique_tools:
    try:
        connection_info = superface.is_tool_connected(user_id=user_id, tool_name=tool)
        is_connected = connection_info.get("connected", False)
        tool_connection_status[tool] = {
            "provider": connection_info.get("provider"),
            "connected": is_connected
        }
        
        if not is_connected:
            all_tools_connected = False
            print(f"ERROR: Tool {tool} is not connected!")
            
    except Exception as e:
        print(f"Error checking connection for {tool}: {str(e)}")
        tool_connection_status[tool] = {
            "provider": None,
            "connected": False,
            "error": str(e)
        }
        all_tools_connected = False

# Stop execution if any tool is not connected
if not all_tools_connected:
    print("\nBenchmark cannot proceed: Some required tools are not connected.")

    # Get configuration url
    try:
        config_url = get_configuration_url(
            user_id=user_id
        )
        print(f"\nPlease visit the following URL to configure your tools:")
        print(f"{config_url}")
    except Exception as e:
        print(f"Error getting configuration URL: {str(e)}")


    results = {
        "status": "failed",
        "reason": "Some required tools are not connected, use the configuration URL to connect them",
        "tool_connection_status": tool_connection_status,
        "configuration_url": config_url
    }
    raise SystemExit("Benchmark stopped due to disconnected tools")

# If we get here, all tools are connected
print("\nAll required tools are connected. Proceeding with benchmark.")

# Compile results
results = {
    "status": "ready",
    "benchmark_data": bench_data,
    "tool_connection_status": tool_connection_status
}

Found 2 unique tools in benchmark data:
- hubspot__list-or-search-engagements__ListOrSearchEngagements
- hubspot__list-or-search-contacts__ListOrSearchContacts

All required tools are connected. Proceeding with benchmark.


In [18]:
# Run the benchmark tests
import time
import json
import statistics
from datetime import datetime

import benchmarks.run_agent
importlib.reload(benchmarks.run_agent)
from benchmarks.run_agent import run

import benchmarks.compare_tool_calls
importlib.reload(benchmarks.compare_tool_calls)
from benchmarks.compare_tool_calls import compare_tool_calls

# Get number of runs per test from environment variable, default to 3
NUM_RUNS_PER_TEST = int(os.getenv("BENCHMARK_RUNS_PER_TEST", 3))

# Function to run a single benchmark test
def run_benchmark_test(test_case, user_id):
    intent = test_case.get('intent')
    expected_tool_calls = test_case.get('expected_tool_calls', [])
    
    print(f"Running test: {intent}")
    print(f"Expected tool calls: {json.dumps(expected_tool_calls, indent=2)}")
    
    # Run the test multiple times
    run_results = []
    test_start_time = time.time()
    overall_success = True
    overall_tool_calls_match = True
    
    for run_num in range(NUM_RUNS_PER_TEST):
        print(f"\n--- Run {run_num + 1}/{NUM_RUNS_PER_TEST} ---")
        
        start_time = time.time()
        try:
            result = run(
                prompt=intent,
                specialist_id="hubspot",
                user_id=user_id,
                model="gpt-4o"
            )
            
            # Extract actual tool calls from the result
            actual_tool_calls = result.get('tool_calls', [])
            
            print(f"Extracted tool calls: {json.dumps(actual_tool_calls, indent=2)}")
            
            # Compare expected and actual tool calls
            tool_calls_match, mismatch_reason = compare_tool_calls(expected_tool_calls, actual_tool_calls)
            success = True
            error = None
            
            # Update overall status
            if not tool_calls_match:
                overall_tool_calls_match = False
                
        except Exception as e:
            result = None
            success = False
            error = str(e)
            tool_calls_match = False
            mismatch_reason = f"Exception occurred: {str(e)}"
            actual_tool_calls = []
            
            # Update overall status
            overall_success = False
            overall_tool_calls_match = False
        
        end_time = time.time()
        duration = end_time - start_time
        
        run_result = {
            "run_number": run_num + 1,
            "actual_tool_calls": actual_tool_calls,
            "tool_calls_match": tool_calls_match,
            "mismatch_reason": mismatch_reason if not tool_calls_match else None,
            "result": result,
            "success": success,
            "error": error,
            "duration_seconds": duration
        }
        run_results.append(run_result)
        
        print(f"Run completed in {duration:.2f} seconds")
        print(f"Success: {success}")
        if not success:
            print(f"Error: {error}")
        
        # Add a small delay between runs
        if run_num < NUM_RUNS_PER_TEST - 1:
            time.sleep(2)
    
    # Calculate statistics across runs
    success_rate = sum(1 for r in run_results if r['success']) / NUM_RUNS_PER_TEST * 100
    tool_call_match_rate = sum(1 for r in run_results if r.get('tool_calls_match', False)) / NUM_RUNS_PER_TEST * 100
    durations = [r['duration_seconds'] for r in run_results]
    avg_duration = statistics.mean(durations) if durations else 0
    min_duration = min(durations) if durations else 0
    max_duration = max(durations) if durations else 0
    std_dev_duration = statistics.stdev(durations) if len(durations) > 1 else 0
    test_duration = time.time() - test_start_time
    
    # Collect mismatch reasons
    mismatch_reasons = [r.get('mismatch_reason') for r in run_results if not r.get('tool_calls_match', True)]
    mismatch_summary = {}
    for reason in mismatch_reasons:
        if reason:
            mismatch_summary[reason] = mismatch_summary.get(reason, 0) + 1
    
    print(f"\n--- Test Statistics ---")
    print(f"Success rate: {success_rate:.2f}%")
    print(f"Tool call match rate: {tool_call_match_rate:.2f}%")
    print(f"Average duration: {avg_duration:.2f} seconds")
    print(f"Min duration: {min_duration:.2f} seconds")
    print(f"Max duration: {max_duration:.2f} seconds")
    print(f"Standard deviation: {std_dev_duration:.2f} seconds")
    
    if mismatch_summary:
        print("\nMismatch reasons:")
        for reason, count in mismatch_summary.items():
            print(f"- {reason} (occurred {count} times)")
    
    return {
        "intent": intent,
        "expected_tool_calls": expected_tool_calls,
        "runs": run_results,
        "stats": {
            "success_rate": success_rate,
            "tool_call_match_rate": tool_call_match_rate,
            "avg_duration": avg_duration,
            "min_duration": min_duration,
            "max_duration": max_duration,
            "std_dev_duration": std_dev_duration,
            "num_runs": NUM_RUNS_PER_TEST,
            "mismatch_summary": mismatch_summary
        },
        # Add top-level properties for easier access
        "success": overall_success,
        "tool_calls_match": overall_tool_calls_match,
        "duration_seconds": test_duration,
        "result": run_results[-1]["result"] if run_results and run_results[-1]["success"] else None,
        "error": None if overall_success else run_results[-1].get("error", "Unknown error")
    }

# Run all benchmark tests
benchmark_results = []
for i, test_case in enumerate(bench_data):
    print(f"\n=== Test {i+1}/{len(bench_data)} ===")
    result = run_benchmark_test(test_case, user_id)
    benchmark_results.append(result)
    print(f"Test completed in {result['duration_seconds']:.2f} seconds")
    print(f"Success: {result['success']}")
    if not result['success']:
        print(f"Error: {result['error']}")
    else:
        # Convert CrewOutput to string first, then truncate if needed
        result_str = str(result['result'])
        if len(result_str) > 200:
            print(f"Result: {result_str[:200]}...")
        else:
            print(f"Result: {result_str}")
    
    # Add a small delay between tests
    if i < len(bench_data) - 1:
        time.sleep(2)

# Compile final results
final_results = {
    "timestamp": datetime.now().isoformat(),
    "benchmark_name": "HubSpot Superface Specialist",
    "total_tests": len(bench_data),
    "successful_tests": sum(1 for r in benchmark_results if r['success']),
    "failed_tests": sum(1 for r in benchmark_results if not r['success']),
    "tool_calls_match_count": sum(1 for r in benchmark_results if r.get('tool_calls_match', False)),
    "results": benchmark_results
}

# For JSON serialization, convert CrewOutput objects to strings
serializable_results = []
for r in benchmark_results:
    serializable_result = r.copy()
    if r['success'] and r['result'] is not None:
        serializable_result['result'] = str(r['result'])
    # Make sure actual_tool_calls is included
    if 'actual_tool_calls' not in serializable_result:
        serializable_result['actual_tool_calls'] = []
    serializable_results.append(serializable_result)

print("\n=== Benchmark Summary ===")
print(f"Total tests: {final_results['total_tests']}")
print(f"Successful tests: {final_results['successful_tests']}")
print(f"Failed tests: {final_results['failed_tests']}")
print(f"Tests with matching tool calls: {final_results['tool_calls_match_count']}")
print(f"Success rate: {(final_results['successful_tests'] / final_results['total_tests']) * 100:.2f}%")
print(f"Tool call match rate: {(final_results['tool_calls_match_count'] / final_results['total_tests']) * 100:.2f}%")


# Save results to file
results_filename = f"./results/hubspot_benchmark_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(results_filename, "w") as f:
    json.dump(final_results, f, indent=2)

print(f"\nResults saved to {results_filename}")


=== Test 1/1 ===
Running test: John needs to retrieve past engagements (calls, emails, meetings) related to Clara Reynolds to prepare the CEO for an important meeting.
Expected tool calls: [
  {
    "tool_name": "hubspot__list-or-search-contacts__ListOrSearchContacts",
    "tool_input": "{\"search\":\"Clara Reynolds\"}",
    "tool_output": {
      "status": "completed"
    }
  },
  {
    "tool_name": "hubspot__list-or-search-engagements__ListOrSearchEngagements",
    "tool_input": "{\"type\": \"CALL\", \"contactId\": \"<CONTACT_ID>\"}",
    "tool_output": {
      "status": "completed"
    }
  },
  {
    "tool_name": "hubspot__list-or-search-engagements__ListOrSearchEngagements",
    "tool_input": "{\"type\": \"EMAIL\", \"contactId\": \"<CONTACT_ID>\"}",
    "tool_output": {
      "status": "completed"
    }
  },
  {
    "tool_name": "hubspot__list-or-search-engagements__ListOrSearchEngagements",
    "tool_input": "{\"type\": \"MEETING\", \"contactId\": \"<CONTACT_ID>\"}",
    "tool_ou

Overriding of current TracerProvider is not allowed


# Agent: Hubspot Agent
## Task: John needs to retrieve past engagements (calls, emails, meetings) related to Clara Reynolds to prepare the CEO for an important meeting.


# Agent: Hubspot Agent
## Thought: I need to retrieve past engagements (calls, emails, meetings) related to Clara Reynolds from Hubspot.
## Using tool: hubspot
## Tool Input: 
"{\"intent\": \"Retrieve engagements for a contact\", \"data\": {\"contact_name\": \"Clara Reynolds\"}}"
## Tool Output: 
{'kind': 'content', 'content': 'Here are the engagements for the contact Clara Reynolds:\n\n- **Call Engagement:**\n  - **Date:** March 20, 2025\n  - **Owner ID:** 77763275\n\nThere were no tasks, notes, or meetings found for this contact. Additionally, I encountered an error while trying to retrieve email engagements due to missing required scopes. If you need further assistance with email engagements, please ensure the necessary permissions are granted.', 'executions': [{'id': 'call_2yV54i7lFQcR1e6ufYe3G2un', 'groupId': 'xO

Overriding of current TracerProvider is not allowed


# Agent: Hubspot Agent
## Task: John needs to retrieve past engagements (calls, emails, meetings) related to Clara Reynolds to prepare the CEO for an important meeting.


# Agent: Hubspot Agent
## Thought: To retrieve past engagements related to Clara Reynolds, I will use Hubspot Tool to search for engagements associated with her contact record in the CRM.
## Using tool: hubspot
## Tool Input: 
"{\"intent\": \"find_engagements\", \"data\": {\"contact_name\": \"Clara Reynolds\"}}"
## Tool Output: 
{'kind': 'content', 'content': 'I found the following engagements for Clara Reynolds:\n\n1. **Call Engagement:**\n   - **ID:** 75381715758\n   - **Type:** CALL\n   - **Timestamp:** 2025-03-20T08:52:29.442Z\n   - **Owner ID:** 77763275\n\nUnfortunately, I encountered an error while trying to retrieve email engagements due to missing required scopes for accessing email data. There were no meeting, note, or task engagements found for Clara Reynolds.', 'executions': [{'id': 'call_SOe3oXx52AGEhs7JF

Overriding of current TracerProvider is not allowed


# Agent: Hubspot Agent
## Task: John needs to retrieve past engagements (calls, emails, meetings) related to Clara Reynolds to prepare the CEO for an important meeting.


# Agent: Hubspot Agent
## Using tool: hubspot
## Tool Input: 
"{\"intent\": \"find engagements\", \"data\": {\"contact_name\": \"Clara Reynolds\"}}"
## Tool Output: 
{'kind': 'content', 'content': "I found the following engagements for Clara Reynolds:\n\n1. **Call Engagement:**\n   - **ID:** 75381715758\n   - **Type:** CALL\n   - **Timestamp:** 2025-03-20T08:52:29.442Z\n   - **Owner ID:** 77763275\n\nUnfortunately, I couldn't retrieve email engagements due to missing required scopes in the API access. There are no meeting, note, or task engagements associated with Clara Reynolds at this time.", 'executions': [{'id': 'call_vPB1Axa4XsVIrZtXkm2kAe5U', 'groupId': 'WxU5GNu', 'toolCall': {'kind': 'function', 'id': 'call_vPB1Axa4XsVIrZtXkm2kAe5U', 'groupId': 'Lqo3dc9', 'name': 'hubspot__list-or-search-contacts__ListOrSearchC